In [1]:
import supereeg as se
import pandas as pd
import numpy as np
import os

In [3]:
ram_data_dir = '/Users/lucyowen/repos/supereeg_paper/data/ram'
pyfr_data_dir = '/Users/lucyowen/repos/supereeg_paper/data/pyfr'

## PyFR analysis

In [5]:
pyfr_across = pd.read_csv('/Users/lucyowen/Desktop/supereeg/supereeg_paper_data/paper_figures/pyfr/pyfr_across.csv', index_col=0)
pyfr_within = pd.read_csv('/Users/lucyowen/Desktop/supereeg/supereeg_paper_data/paper_figures/pyfr/pyfr_within.csv', index_col=0)

In [6]:
pyfr_across.head()

,Unnamed: 0.1,R,Correlation,Subject,Electrode,Sample rate,Samples,Total Electrodes,Sessions,Number thresholded
0,0,[-63.64 -21.93 5.37],0.844742,FR190,7,250.0,3465000,10,7.0,42
1,0,[-48.48 38.4 17.26],0.861268,CH013,34,250.0,1549237,121,2.0,2
2,0,[ 49.49 -34.7 35.17],0.665390,CH048,36,250.0,2454000,44,4.0,79
3,0,[ 65.66 -67.29 25.83],0.641038,TJ019,62,250.0,3600000,82,2.0,16
4,0,[-36.36 -1.81 59.76],0.764453,FR050,51,250.0,447750,98,1.0,2


In [7]:
pyfr_within.head()

,Unnamed: 0.1,R,Correlation,Full,Subject
0,0,[ 61.62 -13.33 -25.8 ],0.310237,TJ032_62_within,TJ032
1,0,[ 63.64 -70.63 31.1 ],0.665180,FR150_52_within,FR150
2,0,[-56.57 0.16 -27.4 ],0.683088,CH068_2_within,CH068
3,0,[-55.56 35.11 21.44],0.371791,CH067_37_within,CH067
4,0,[62.63 -2.89 39.03],0.443610,FR220_28_within,FR220


In [8]:
pyfr_within.shape

(4170, 5)

In [9]:
pyfr_across.shape

(4170, 10)

## Merge dataframes

In [10]:
pyFR_df_m = pd.merge(pyfr_across, pyfr_within, on=['Subject', 'R'])
pyFR_df_m.dropna(inplace=True)
pyFR_df_m.rename(columns={"Correlation_x":"Correlation_Across", "Correlation_y":"Correlation_Within"}, inplace=True)
pyFR_df = pd.DataFrame()
pyFR_df['across'] = pyFR_df_m['Correlation_Across']
pyFR_df['within'] = pyFR_df_m['Correlation_Within']
pyFR_df['Subject'] = pyFR_df_m['Subject']
pyFR_df['R'] = pyFR_df_m['R']
pyFR_df['Sample rate'] = pyFR_df_m['Sample rate']
pyFR_df['Samples'] = pyFR_df_m['Samples']
pyFR_df['Thresholded electrodes'] = pyFR_df_m['Total Electrodes']
pyFR_df['Sessions'] = pyFR_df_m['Sessions']
pyFR_df['Number thresholded'] = pyFR_df_m['Number thresholded']

## With a few other stats

In [10]:
pyFR_df['Total time (h)'] = np.round(pyFR_df['Samples']/pyFR_df['Sample rate']/3600,3)
pyFR_df['Average session length (h)'] = np.round(pyFR_df['Total time (h)']/pyFR_df['Sessions'],3)
pyFR_df['Average session length (h)'].head()
pyFR_df['Total electrodes'] = pyFR_df['Thresholded electrodes'] + pyFR_df['Number thresholded']
pyFR_df['Proportion thresholded'] = np.round(pyFR_df['Number thresholded'] / pyFR_df['Total electrodes'], 3)

## Rename subject ids 

In [11]:
grouped_results_pyFR_df = pyFR_df.groupby('Subject')['within', 'across', 'Sample rate', 'Samples', 
                                                  'Sessions', 'Number thresholded', 'Total time (h)', 'Average session length (h)', 
                                                  'Total electrodes', 'Proportion thresholded'].mean()

grouped_results_pyFR_df['Subject'] = range(grouped_results_pyFR_df.shape[0])
subj_dict = grouped_results_pyFR_df['Subject'].to_dict()
pyFR_df.Subject = [subj_dict[item] for item in pyFR_df.Subject] 

In [12]:
pyFR_df.to_csv(os.path.join(pyfr_data_dir, 'pyfr.csv'))

# RAM

In [13]:
W_a_a = pd.read_csv('/Users/lucyowen/Desktop/supereeg/supereeg_paper_data/paper_figures/ram/all/a_sub-a_exp.csv', index_col=0)
W_a_a.name = 'W_a_a'
W_a_all = pd.read_csv('/Users/lucyowen/Desktop/supereeg/supereeg_paper_data/paper_figures/ram/all/a_sub-all_exp.csv', index_col=0)
W_a_all.name = 'W_a_all'
W_a_w = pd.read_csv('/Users/lucyowen/Desktop/supereeg/supereeg_paper_data/paper_figures/ram/all/a_sub-w_exp.csv', index_col=0)
W_a_w.name = 'W_a_w'
W_w_a = pd.read_csv('/Users/lucyowen/Desktop/supereeg/supereeg_paper_data/paper_figures/ram/all/w_sub-a_exp.csv', index_col=0)
W_w_a.name = 'W_w_a'
W_w_all = pd.read_csv('/Users/lucyowen/Desktop/supereeg/supereeg_paper_data/paper_figures/ram/all/w_sub-all_exp.csv', index_col=0)
W_w_all.name = 'W_w_all'
W_w_w = pd.read_csv('/Users/lucyowen/Desktop/supereeg/supereeg_paper_data/paper_figures/ram/all/w_sub-w_exp.csv', index_col=0)
W_w_w.name = 'W_w_w'

In [14]:
dfs = [W_a_a, W_a_all, W_a_w, W_w_a, W_w_all, W_w_w]

In [22]:
RAM_df = dfs[0]
RAM_df = RAM_df.rename(columns={"Correlation":RAM_df.name})
for df_ in dfs[1:]:
    RAM_df = RAM_df.merge(df_, on=['Subject', 'R', 'Electrode', 'Samples', 'Total Electrodes','Sample rate','Sessions', 'Number thresholded'])
    
    RAM_df = RAM_df.rename(columns={"Correlation":df_.name})

In [23]:
RAM_df.dropna(inplace=True)

In [17]:
RAM_df['Sessions'] = RAM_df['Sessions'] + 1

In [18]:
RAM_df['Thresholded electrodes'] = RAM_df['Total Electrodes']
RAM_df['Total time (h)'] = np.round(RAM_df['Samples']/RAM_df['Sample rate']/3600,3)
RAM_df['Average session length (h)'] = np.round(RAM_df['Total time (h)']/RAM_df['Sessions'],3)
RAM_df['Average session length (h)'].head()
RAM_df['Total electrodes'] = RAM_df['Thresholded electrodes'] + RAM_df['Number thresholded']
RAM_df['Proportion thresholded'] = np.round(RAM_df['Number thresholded'] / RAM_df['Total electrodes'], 3)

In [21]:
all_subject_e = np.array([])
all_exp_e = np.array([])
for i , item in enumerate(RAM_df['Subject'].values):
    all_subject_e = np.append(all_subject_e, item.split('_')[0])
    all_exp_e = np.append(all_exp_e, item.split('_')[1]+ '_' + item.split('_')[2])

In [22]:
RAM_df['Subject'] = all_subject_e
RAM_df['Experiment'] = all_exp_e

In [23]:
grouped_results_ram = RAM_df.groupby('Subject')['Experiment','W_w_w', 'W_w_a', 'W_w_all', 'W_a_w', 'W_a_a','W_a_all', 'Sample rate', 'Samples', 
                                                  'Sessions', 'Number thresholded', 'Total time (h)', 'Average session length (h)', 
                                                  'Total electrodes', 'Proportion thresholded'].mean()

grouped_results_ram['Subject'] = range(grouped_results_ram.shape[0])
subj_dict = grouped_results_ram['Subject'].to_dict()
RAM_df.Subject = [subj_dict[item] for item in RAM_df.Subject] 

In [24]:
grouped_results_ram['Subject'].index

Index(['R1032D', 'R1061T', 'R1075J', 'R1083J', 'R1092J', 'R1094T', 'R1106M',
       'R1108J', 'R1111M', 'R1112M', 'R1135E', 'R1147P', 'R1167M', 'R1169P',
       'R1171M', 'R1174T', 'R1189M', 'R1215M', 'R1221P', 'R1243T', 'R1260D',
       'R1264P', 'R1274T', 'R1291M'],
      dtype='object', name='Subject')

In [26]:
RAM_df.to_csv(os.path.join(ram_data_dir, 'ram.csv'))

# Frequency analysis